In [13]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
import time
import numpy as np # NDCG 계산 시 필요

# --- 1. 데이터 로드 ---
# 'review_business_5up_5aspect_3sentiment_vectorized_clean.json' 파일 경로를 지정하세요.
# 파일이 현재 스크립트와 같은 디렉터리에 있다고 가정합니다.
file_path = 'review_business_5up_5aspect_3sentiment_vectorized_clean.json'
df = pd.read_json(file_path, lines=True)

# 필요한 컬럼만 선택: user_id, business_id, stars (평점)
df_ratings = df[['user_id', 'business_id', 'stars']]

# Surprise Reader 객체 생성
# rating_scale은 평점 척도 (여기서는 1점부터 5점까지)
reader = Reader(rating_scale=(1, 5))

# DataFrame으로부터 Surprise Dataset 로드
data = Dataset.load_from_df(df_ratings, reader)

# --- 2. 학습셋과 테스트셋 분리 ---
# 넷플릭스 대회처럼 (80% 학습, 20% 테스트)
# random_state를 고정하여 매번 동일한 결과가 나오도록 합니다.
trainset_surprise, testset_surprise = train_test_split(data, test_size=0.2, random_state=42)

# 학습셋과 테스트셋의 크기 출력 (Trainset 객체의 n_ratings 속성 사용)
print(f"학습셋 크기: {trainset_surprise.n_ratings}개")
print(f"테스트셋 크기: {len(testset_surprise)}개")

학습셋 크기: 358236개
테스트셋 크기: 89560개


In [14]:
from collections import defaultdict
import numpy as np

# Top-N 추천 목록 생성 함수
def get_top_n(predictions, n=10):
    """
    예측 결과를 바탕으로 각 사용자에 대한 상위 N개 추천 목록을 반환합니다.

    Args:
        predictions: Surprise 모델의 예측 결과 리스트 (uid, iid, r_ui, est, details)
        n: 반환할 상위 아이템 수

    Returns:
        dict: {user_id: [(item_id, predicted_rating), ...]} 형태의 딕셔너리
    """
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True) # 예측 평점(est) 기준으로 내림차순 정렬
        top_n[uid] = user_ratings[:n] # 상위 n개만 유지
    return top_n

# Precision, Recall, NDCG 계산 함수
def calculate_metrics(predictions, testset, k=10, relevant_threshold=4.0):
    """
    추천 목록의 Precision, Recall, NDCG를 계산합니다.

    Args:
        predictions: 모델의 예측 결과 리스트 (uid, iid, r_ui, est, details)
        testset: Surprise 테스트 세트 (튜플 리스트: (uid, iid, true_rating))
        k: Top-K 추천 수
        relevant_threshold: 관련성 있는 아이템을 정의하는 평점 임계값 (예: 4.0 이상)

    Returns:
        tuple: (precision_mean, recall_mean, ndcg_mean) 평균 값
    """
    # 1. 각 사용자별 실제 관련성 있는 아이템 셋 생성
    user_relevant_items = defaultdict(set)
    for uid, iid, true_r in testset:
        if true_r >= relevant_threshold:
            user_relevant_items[uid].add(iid)

    # 2. 모델의 Top-K 추천 목록 생성
    user_top_k_recs = get_top_n(predictions, n=k)

    precisions = []
    recalls = []
    ndcgs = []

    # 각 사용자별로 Precision, Recall, NDCG 계산
    for uid, recommended_items in user_top_k_recs.items():
        if uid not in user_relevant_items or not user_relevant_items[uid]:
            continue
        
        relevant_in_k = 0
        for iid, _ in recommended_items:
            if iid in user_relevant_items[uid]:
                relevant_in_k += 1
        
        precision = relevant_in_k / k if k > 0 else 0
        precisions.append(precision)

        num_true_relevant = len(user_relevant_items[uid])
        recall = relevant_in_k / num_true_relevant if num_true_relevant > 0 else 0
        recalls.append(recall)

        dcg = 0.0
        num_ideal_relevant_in_testset = len(user_relevant_items[uid])
        
        for i, (iid, _) in enumerate(recommended_items):
            relevance = 1.0 if iid in user_relevant_items[uid] else 0.0
            dcg += relevance / np.log2(i + 2)

        idcg = 0.0
        for i in range(min(num_ideal_relevant_in_testset, k)):
            idcg += 1.0 / np.log2(i + 2)

        ndcg = dcg / idcg if idcg > 0 else 0.0
        ndcgs.append(ndcg)

    return np.mean(precisions) if precisions else 0, \
           np.mean(recalls) if recalls else 0, \
           np.mean(ndcgs) if ndcgs else 0

In [15]:
from surprise import KNNWithMeans, SVD
from surprise import accuracy # RMSE, MAE 계산을 위해 필요

print("--- 협업 필터링 (KNNWithMeans) 및 SVD 모델 학습 및 평가 시작 ---")

# --- UBCF (User-Based Collaborative Filtering) ---
print("\n--- UBCF (KNNWithMeans, 이웃 수 100, cosine 유사도) 모델 학습 및 평가 ---")
# user_based=True로 설정하여 UBCF 사용
ubcf_algo_k100 = KNNWithMeans(k=100, sim_options={'name': 'cosine', 'user_based': True}, random_state=42)
start_time = time.time()
ubcf_predictions = ubcf_algo_k100.fit(trainset_surprise).test(testset_surprise)
end_time = time.time()
print(f"UBCF 모델 예측 시간: {end_time - start_time:.2f}초")

# RMSE, MAE 계산
ubcf_rmse = accuracy.rmse(ubcf_predictions, verbose=False)
ubcf_mae = accuracy.mae(ubcf_predictions, verbose=False)

# Precision, Recall, NDCG 계산
precision_ubcf, recall_ubcf, ndcg_ubcf = calculate_metrics(ubcf_predictions, testset_surprise, k=10)

print(f"UBCF RMSE: {ubcf_rmse:.4f}")
print(f"UBCF MAE: {ubcf_mae:.4f}")
print(f"UBCF Precision@10: {precision_ubcf:.4f}")
print(f"UBCF Recall@10: {recall_ubcf:.4f}")
print(f"UBCF NDCG@10: {ndcg_ubcf:.4f}")

# --- IBCF (Item-Based Collaborative Filtering) ---
print("\n--- IBCF (KNNWithMeans, 이웃 수 100, cosine 유사도) 모델 학습 및 평가 ---")
# user_based=False로 설정하여 IBCF 사용
ibcf_algo_k100 = KNNWithMeans(k=100, sim_options={'name': 'cosine', 'user_based': False}, random_state=42)
start_time = time.time()
ibcf_predictions = ibcf_algo_k100.fit(trainset_surprise).test(testset_surprise)
end_time = time.time()
print(f"IBCF 모델 예측 시간: {end_time - start_time:.2f}초")

# RMSE, MAE 계산
ibcf_rmse = accuracy.rmse(ibcf_predictions, verbose=False)
ibcf_mae = accuracy.mae(ibcf_predictions, verbose=False)

# Precision, Recall, NDCG 계산
precision_ibcf, recall_ibcf, ndcg_ibcf = calculate_metrics(ibcf_predictions, testset_surprise, k=10)

print(f"IBCF RMSE: {ibcf_rmse:.4f}")
print(f"IBCF MAE: {ibcf_mae:.4f}")
print(f"IBCF Precision@10: {precision_ibcf:.4f}")
print(f"IBCF Recall@10: {recall_ibcf:.4f}")
print(f"IBCF NDCG@10: {ndcg_ibcf:.4f}")

# --- SVD 모델 학습 및 평가 (n_epochs=20 적용) ---
print("\n--- SVD 모델 학습 및 평가 (n_epochs=20 적용) ---")
# n_factors=1, n_epochs=20, lr_all=0.005, reg_all=0.02
svd_algo = SVD(n_factors=1, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
start_time = time.time()
svd_predictions = svd_algo.fit(trainset_surprise).test(testset_surprise)
end_time = time.time()
print(f"SVD 모델 예측 시간: {end_time - start_time:.2f}초")

# RMSE, MAE 계산
svd_rmse = accuracy.rmse(svd_predictions, verbose=False)
svd_mae = accuracy.mae(svd_predictions, verbose=False)

# Precision, Recall, NDCG 계산
precision_svd, recall_svd, ndcg_svd = calculate_metrics(svd_predictions, testset_surprise, k=10)

print(f"SVD RMSE: {svd_rmse:.4f}")
print(f"SVD MAE: {svd_mae:.4f}")
print(f"SVD Precision@10: {precision_svd:.4f}")
print(f"SVD Recall@10: {recall_svd:.4f}")
print(f"SVD NDCG@10: {ndcg_svd:.4f}")

--- 협업 필터링 (KNNWithMeans) 및 SVD 모델 학습 및 평가 시작 ---

--- UBCF (KNNWithMeans, 이웃 수 100, cosine 유사도) 모델 학습 및 평가 ---
Computing the cosine similarity matrix...
Done computing similarity matrix.
UBCF 모델 예측 시간: 40.25초
UBCF RMSE: 1.0905
UBCF MAE: 0.8274
UBCF Precision@10: 0.2590
UBCF Recall@10: 0.9830
UBCF NDCG@10: 0.9470

--- IBCF (KNNWithMeans, 이웃 수 100, cosine 유사도) 모델 학습 및 평가 ---
Computing the cosine similarity matrix...
Done computing similarity matrix.
IBCF 모델 예측 시간: 4.91초
IBCF RMSE: 1.0774
IBCF MAE: 0.8165
IBCF Precision@10: 0.2593
IBCF Recall@10: 0.9832
IBCF NDCG@10: 0.9491

--- SVD 모델 학습 및 평가 (n_epochs=20 적용) ---
SVD 모델 예측 시간: 1.84초
SVD RMSE: 1.0366
SVD MAE: 0.8079
SVD Precision@10: 0.2593
SVD Recall@10: 0.9832
SVD NDCG@10: 0.9523
